# Spark and MLib using a dataset published by AudioScrobbler

To build a recommendation system using Spark and MLib using a dataset published by AudioScrobbler. This data is 500MB uncompressed and can be downloaded here- http://www-etud.iro.umontreal.ca/%7Ebergstrj/audioscrobbler_data.html



### Preparing the data

##### Importing user_artist_data.txt
Breaking the data into smaller pieces—more partitions—for processing. This can let put more processor cores to work on the problem at once. Supplying this as a second argument to this method to specify a different and larger number of partitions.
Setting the minimal number of partitions that will be created, while reading the file, by setting it in the optional parameter minPartitions of textFile.

In [1]:
rawUserArtistData = sc.textFile("audio-data/user_artist_data.txt",minPartitions=6)
rawUserArtistData.cache()
print "length of user_artist_data.txt :",rawUserArtistData.count()
print rawUserArtistData.take(10)

length of user_artist_data.txt : 24296858
[u'1000002 1 55', u'1000002 1000006 33', u'1000002 1000007 8', u'1000002 1000009 144', u'1000002 1000010 314', u'1000002 1000013 8', u'1000002 1000014 42', u'1000002 1000017 69', u'1000002 1000024 329', u'1000002 1000025 1']


Each line of the file contains a user ID, an artist ID, and a play count, separated by spaces. To compute statistics on the user ID, we split the line by space, and the first (0-indexed) value is parsed as a number. The stats() method returns an object containing statistics like maximum and minimum.

In [3]:
print rawUserArtistData.map(lambda user: float(user.split()[0])).stats()

(count: 24296858, mean: 1947573.26535, stdev: 496000.544975, max: 2443548.0, min: 90.0)


In [4]:
print rawUserArtistData.map(lambda artist: float(artist.split()[1])).stats()

(count: 24296858, mean: 1718704.09376, stdev: 2539389.04017, max: 10794401.0, min: 1.0)


The computed statistics that are printed reveal that the maximum user and artist IDs are 2443548 and 10794401, respectively. These are comfortably smaller than 2147483647. No additional transformation will be necessary to use these IDs.

Taking the artist names corresponding to the opaque numeric IDs contained in artist_data.txt. It contains the artist ID and name separated by a tab.

In [2]:
rawArtistData = sc.textFile("audio-data/artist_data.txt")
head1 = rawArtistData.take(10)
print head1

[u'1134999\t06Crazy Life', u'6821360\tPang Nakarin', u'10113088\tTerfel, Bartoli- Mozart: Don', u'10151459\tThe Flaming Sidebur', u'6826647\tBodenstandig 3000', u'10186265\tJota Quest e Ivete Sangalo', u'6828986\tToto_XX (1977', u'10236364\tU.S Bombs -', u'1135000\tartist formaly know as Mat', u'10299728\tKassierer - Musik f\xfcr beide Ohren']


A small number of the lines appear to be corrupted. They don’t contain a tab, or they inadvertently include a newline character. These lines cause a NumberFormatException, and ideally, they would not mapto anything at all. However, the map() function must return exactly one value for every input, so it can’tbe used. It’s possible to remove the lines that don’t parse with filter(), but this would duplicate the parsing logic. The flatMap() function is appropriate when each element maps to zero, one, or more results, because it simply “flattens” these collections of zero or more results from each input into one big RDD.

In [3]:
def parser_artist(line):
    
    #code to parse artist_data into lists [id, artist]
    tokens = line.split('\t')
    
    if len(tokens) == 2:
        try:
            id_ = int(tokens[0])
            name_ = tokens[1]
            return [(id_,name_)]
        except:
            return [] 
    else:
        return []

In [4]:
artistByID = dict(rawArtistData.flatMap(lambda x: parser_artist(x)).collect())

Checking few artist ID's

In [5]:
artistByID[1134999]

u'06Crazy Life'

In [6]:
artistByID[6821360]

u'Pang Nakarin'

##### Importing artist_alias.txt

In [7]:
rawArtistAlias = sc.textFile('audio-data/artist_alias.txt')
head2 = rawArtistAlias.take(10)
print head2

[u'1092764\t1000311', u'1095122\t1000557', u'6708070\t1007267', u'10088054\t1042317', u'1195917\t1042317', u'1112006\t1000557', u'1187350\t1294511', u'1116694\t1327092', u'6793225\t1042317', u'1079959\t1000557']


The artist_alias.txt file maps artist IDs that may be misspelled or nonstandard to the ID of the artist’s canonical name. It contains two IDs per line, separated by a tab. This file is relatively small, containing about 200,000 entries. It will be useful to collect it as a Map, mapping “bad” artist IDs to “good” ones, instead of just using it as an RDD of pairs of artist IDs. Again, some lines are missing the first artist ID, for some reason, and are skipped.

In [8]:
def parser_alias(line):
    
    tokens = line.split('\t')
    
    if len(tokens) == 2:
        try:
            id_ = int(tokens[0])
            name_ = int(tokens[1])
            return [(id_,name_)]
        except:
            return [] 
    else:
        return []



In [9]:
artistAlias = rawArtistAlias.flatMap(lambda x: parser_alias(x)).collectAsMap()

The first entry, for instance, maps ID 6803336 to 1000010. We can look these up from the RDD containing artist names:

In [10]:
artistAlias[6803336]

1000010

In [11]:
artistByID[6803336]

u'Aerosmith (unplugged)'

In [12]:
artistByID[1000010]

u'Aerosmith'

This entry evidently maps “Aerosmith (unplugged)” to “Aerosmith.

### <b>Building the model

Although the data set is in nearly the right form for use with pySpark MLlib’s ALS implementation, it requires two small extra transformations. First, the aliases data set should be applied to convert all artist IDs to a canonical ID, if a different canonical ID exists. Second, the data should be converted into Rating objects, which is the imple‐ mentation’s abstraction for user-product-value data. Despite the name, Rating is suitable for use with implicit data. Note also that MLlib refers to “products” throughout its API, and so will this example, but the “products” here are artists. The underlying model is not at all specific to recommending products, or for that matter, to recommending things to people

In [27]:
#converting all artist IDs to a canonical ID
from pyspark.mllib import recommendation
from pyspark.mllib.recommendation import *

bArtistAlias = sc.broadcast(artistAlias)

def extract(x):
    userID, artistID, count = map(lambda line: int(line), x.split())
    #get artist’s alias if it exists, else get original artist
    finalArtistID = bArtistAlias.value.get(artistID)
    if finalArtistID == None:
        finalArtistID = artistID
    
    return Rating(userID, finalArtistID, count)

trainData = rawUserArtistData.map(lambda x: extract(x))
trainData.cache()

PythonRDD[280] at RDD at PythonRDD.scala:43

The artistAlias mapping created earlier can be referenced directly in an RDD’s map() function, even though it is a local Map on the driver. This works, because it will be copied automatically with every task. However, it is not tiny, consuming about 15 megabytes in memory and at least several megabytes in serialized form. Because many tasks execute in one JVM, it’s wasteful to send and store so many copies of the data. Instead, we create a broadcast variable called bArtistAlias for artistAlias. This makes Spark send and hold in memory just one copy for each executor in the cluster. When there are thousands of tasks, and many execute in parallel on each executor, this can save significant network traffic and memory.

The call to cache() suggests to Spark that this RDD should be temporarily stored after being computed, and furthermore, kept in memory in the cluster. This is helpful because the ALS algorithm is iterative, and will typically need to access this data 10 times or more. Without this, the RDD could be repeatedly recomputed from the original data each time it is accessed!

In [28]:
#building final model with book parameters
model = ALS.trainImplicit(trainData,10 ,5 , 0.01, seed=42)
print "Built model"

Built model


This constructs model as a MatrixFactorizationModel. The operation will likely take minutes or more depending on your cluster. Compared to some machine learning models, whose final form may consist of just a few parameters or coefficients, this type of model is huge. It contains a feature vector of 10 values for each user and product in the model, and in this case there are more than 1.7 million of them. The model contains these large user-feature and product-feature matrices as RDDs of their own.

### <b> Spot Checking Recommendations

We should first see if the artist recommendations make any intuitive sense, by examining a user, his or her plays, and recommendations for that user. Take, for example,user 2093760. Extract the IDs of artists that this user has listened to and print their names. This means searching the input for artist IDs for this user, and then filtering the set of artists by these IDs so you can collect and print the names in order

In [ ]:
bartistByID = sc.broadcast(artistByID)

# Find lines whose user is 2093760, collect unique artists, filter in those artists, get just artist, and print
existingArtistsForUser = (trainData
                  .filter(lambda observation: observation.user == 2093760)
                  .map(lambda observation: bartistByID.value.get(observation.product))
                  .collect())

In [24]:
for artist in existingArtistsForUser:
    print artist

David Gray
The Saw Doctors
Blackalicious
Jurassic 5
Xzibit


The artists look like a mix of mainstream pop and hip-hop. A Jurassic 5 fan? Remember, it’s 2005. Also, the Saw Doctors are a very Irish rock band popular in Ireland.


Making ten recommendations for this user:

In [25]:
recommendationsForUser = \
    map(lambda observation: artistByID.get(observation.product), model.call("recommendProducts", 2093760, 10))

In [26]:
print "Top 10 recommendations for the user 2093760 :"
print ""
for recommend in recommendationsForUser:
    print recommend

Top 10 recommendations for the user 2093760 :

Jay-Z
Kanye West
Snoop Dogg
Nas
50 Cent
2Pac
Outkast
Dr. Dre
Eminem
The Roots


The result is a mix of pop punk and metal. While, this doesn’t look like a great set of recommendations as it lists popular artists and is not personalized to this user’s listening habits, we can improve using hold-out sets and tuning hyper parameters.